# LangGraph Agent v3 — Complete (Analyze → Search → Enrich → Process)

Final, self-contained agent with LLM decision to enrich or process.

Quick guide:
- State: messages, search_results, optimized_query, decision
- Flow: analyze → search → decide → (enrich | process) → process → END
- Observe prints: [ANALYZE], [SEARCH], [DECIDE], [ROUTE], [ENRICH], [PROCESS], [END]
- The decision is persisted in state.decision and printed after run

In [ ]:
import os
from typing import List, Optional, Union, Literal
from dotenv import load_dotenv
from pydantic import BaseModel, Field

from langgraph.graph import StateGraph, END

from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain_community.utilities import GoogleSearchAPIWrapper

from langchain_community.document_loaders import WebBaseLoader

load_dotenv()

In [ ]:
class SearchResult(BaseModel):
    id: int
    title: str
    snippet: str
    link: str
    full_content: Optional[str] = None

class AgentState(BaseModel):
    messages: List[Union[HumanMessage, AIMessage, SystemMessage]] = Field(description="Chat history")
    search_results: List[SearchResult] = Field(default_factory=list)
    optimized_query: Optional[str] = None
    decision: Optional[Literal['enrich','process']] = None

    class Config:
        arbitrary_types_allowed = True

In [ ]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", temperature=0.7)
search_engine = GoogleSearchAPIWrapper()

In [ ]:
def fetch_full_content(url: str, max_length: int = 8000) -> str:
    try:
        docs = WebBaseLoader(web_paths=[url], header_template={"User-Agent":"Mozilla/5.0"}).load()
        if docs and docs[0].page_content.strip():
            t = docs[0].page_content
            return t[:max_length] + ('…' if len(t) > max_length else '')
        return '[Full content unavailable. Using snippet]'
    except Exception as e:
        return f'[Content unavailable: {e}]'

In [ ]:
def analyze_query(state: AgentState):
    print("[ANALYZE] Optimizing user query…")
    user_query = next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    prompt = ChatPromptTemplate.from_messages([
        ("system", "You rewrite user queries into high-recall web search queries. Today's date is 2025-08-27. Preserve key entities and constraints (dates, names, locations). Add helpful synonyms if useful. Return ONLY the optimized query with no quotes or extra text."),
        ("human", "Optimize: {q}")
    ])
    q = llm.invoke(prompt.format_messages(q=user_query)).content.strip().replace('"','')
    return {"optimized_query": q}

In [ ]:
def perform_search(state: AgentState):
    print("[SEARCH] Running web search…")
    query = state.optimized_query or next((m.content for m in reversed(state.messages) if isinstance(m, HumanMessage)), "")
    try:
        raw = search_engine.results(query, num_results=5)
    except Exception:
        raw = [{"title":"Search unavailable","snippet":f"Could not search: {query}","link":"https://example.com"}]
    results = [SearchResult(id=i+1,
                            title=r.get('title','No title'),
                            snippet=r.get('snippet', r.get('description','No content')),
                            link=r.get('link', r.get('url','No link')))
               for i, r in enumerate(raw)]
    return {"search_results": results}

In [ ]:
def enrich_content(state: AgentState):
    print("[ENRICH] Fetching and extracting relevant page content…")
    if not state.search_results:
        return {}
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')
    for r in state.search_results:
        raw = fetch_full_content(r.link)
        print(raw)
        if raw.startswith('[Content unavailable'):
            r.full_content = raw
        else:
            print("-------")
            print(raw)
            print("-------")
            p = ChatPromptTemplate.from_messages([
                ("system", "From the page content, extract only information directly relevant to the user's query. Prefer concrete facts, figures, quotes, and timelines. Include brief inline citations with the source URL (e.g., [source: <URL>]). Ignore irrelevant sections."),
                ("human", "Query: {q}\nTitle: {t}\nURL: {u}\nContent:\n{c}")
            ])
            r.full_content = llm.invoke(p.format_messages(q=user_query, t=r.title, u=r.link, c=raw)).content.strip()
    return {"search_results": state.search_results}

In [ ]:

def process_results(state: AgentState):
    print("[PROCESS] Synthesizing final answer…")
    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')
    if state.search_results:
        blocks = []
        for r in state.search_results:
            snippet = r.snippet or ''
            detail = r.full_content
            blocks.append(f"Result {r.id} | {r.title}\n{snippet}\nSource: {r.link}\n{detail}")
        res = '\n'.join(blocks)
    else:
        res = 'No search results available.'
    prompt = ChatPromptTemplate.from_messages([
        ('system', 'Answer the query using only the provided results. Be concise, precise, and cite URLs inline (e.g., [source: <URL>]). If evidence is weak or conflicting, say so and avoid speculation.'),
        ('human', 'Query: {q}\n\nResults:\n{res}')
    ])
    ans = llm.invoke(prompt.format_messages(q=user_query, res=res)).content.strip()
    return {"messages": [AIMessage(content=ans)]}

In [ ]:
def route_to_search(state: AgentState) -> Literal['search']:
    print("[ROUTE] analyze → search")
    return 'search'

def decide_next(state: AgentState):
    """LLM-backed decision node that writes the decision into state."""
    print("[DECIDE] Choosing 'enrich' vs 'process'…")

    user_query = next((m.content for m in state.messages if isinstance(m, HumanMessage)), 'Unknown')

    p = ChatPromptTemplate.from_messages([
        ('system', "Today is 2025-08-27. Reply ONLY 'enrich' or 'process'. Choose 'process' only if the current snippets clearly and fully answer the query with sufficient detail and sources. Otherwise choose 'enrich'. Be conservative."),
        ('human', 'Query: {q}\nSnippets:\n' + '\n'.join([f'- {r.title}: {r.snippet}' for r in state.search_results]))
    ])
    try:
        d = llm.invoke(p.format_messages(q=user_query)).content.strip().lower()
        decision = 'process' if ('process' in d and 'enrich' not in d) else 'enrich'
    except Exception:
        decision = 'enrich'
    print(f"[DECIDE] Decision: {decision}")
    return {"decision": decision}

def route_by_decision(state: AgentState) -> Literal['enrich','process']:
    nxt = state.decision or 'enrich'
    print(f"[ROUTE] decide → {nxt}")
    return nxt

def is_done(state: AgentState) -> Literal['end']:
    print("[END] process → END")
    return 'end'

In [ ]:
workflow = StateGraph(AgentState)

complete_agent = workflow.compile()

In [ ]:
# Test v3
query = 'Who won the 2024 US presidential election?'
state = AgentState(messages=[HumanMessage(content=query)])
final_state = complete_agent.invoke(state)



In [ ]:
for m in final_state['messages']:
    if isinstance(m, HumanMessage):
        print(f'Human: {m.content}')
    elif isinstance(m, AIMessage):
        print(f'AI: {m.content}')
    else:
        print(m)

In [ ]:
final_state